In [1]:
import pandas as pd
import glob
import os

In [2]:
KENPOM_DATA_DIR = "/Users/jameswhedbee/jupyter_notebooks/DATA/MarchMadness/KenPom"
KAGGLE_DATA_DIR = "/Users/jameswhedbee/jupyter_notebooks/DATA/MarchMadness/MDataFiles_Stage2"
OUTPUT_DATA_DIR = "/Users/jameswhedbee/jupyter_notebooks/DATA/MarchMadness/output"

In [3]:
KENPOM_COLS = ["Season", "TeamName", "AdjOE", "AdjDE"]

In [4]:
all_files = glob.glob(os.path.join(KENPOM_DATA_DIR, "*.csv"))

data = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    data.append(df)

kenpom_df = pd.concat(data, axis=0, ignore_index=True)[KENPOM_COLS]
kenpom_df["LowerTeamName"] = kenpom_df["TeamName"].str.lower()
kenpom_df

,Season,TeamName,AdjOE,AdjDE,LowerTeamName
0,2013,Air Force,112.8840,104.4400,air force
1,2013,Akron,107.8010,94.0583,akron
2,2013,Alabama,104.1100,93.3046,alabama
3,2013,Alabama A&M,90.5796,111.4980,alabama a&m
4,2013,Alabama St.,92.2111,112.0520,alabama st.
...,...,...,...,...,...
6182,2004,Wright St.,103.6850,104.8470,wright st.
6183,2004,Wyoming,104.4840,99.8136,wyoming
6184,2004,Xavier,110.7610,92.9239,xavier
6185,2004,Yale,99.3150,103.3250,yale


In [5]:
teams_df = pd.read_csv(f"{KAGGLE_DATA_DIR}/MTeams.csv")
spellings_df = pd.read_csv(f"{KAGGLE_DATA_DIR}/MTeamSpellings.csv")
teams_df

,TeamID,TeamName,FirstD1Season,LastD1Season
0,1101,Abilene Chr,2014,2021
1,1102,Air Force,1985,2021
2,1103,Akron,1985,2021
3,1104,Alabama,1985,2021
4,1105,Alabama A&M,2000,2021
...,...,...,...,...
366,1467,Merrimack,2020,2021
367,1468,Bellarmine,2021,2021
368,1469,Dixie St,2021,2021
369,1470,Tarleton St,2021,2021


In [6]:
merged_df = kenpom_df.merge(spellings_df, left_on="LowerTeamName", right_on="TeamNameSpelling")[KENPOM_COLS + ["TeamID"]]
merged_df

,Season,TeamName,AdjOE,AdjDE,TeamID
0,2013,Air Force,112.8840,104.4400,1102
1,2011,Air Force,105.7170,101.1380,1102
2,2003,Air Force,105.3400,103.7220,1102
3,2019,Air Force,100.4310,107.2750,1102
4,2007,Air Force,116.2860,97.0562,1102
...,...,...,...,...,...
5912,2017,Fort Wayne,109.5300,108.0540,1236
5913,2018,Fort Wayne,107.6160,106.4470,1236
5914,2021,Bellarmine,107.4090,108.6840,1468
5915,2021,Merrimack,90.8608,101.7180,1467


In [7]:
missing_teams = set(kenpom_df["TeamName"]) - set(merged_df["TeamName"])
missing_teams

{'Arkansas Little Rock',
 'Arkansas Pine Bluff',
 'Bethune Cookman',
 'Cal St. Bakersfield',
 'Dixie St.',
 'Illinois Chicago',
 'LIU',
 'Louisiana Lafayette',
 'Louisiana Monroe',
 'Maryland Eastern Shore',
 'Mississippi Valley St.',
 'Southeast Missouri St.',
 'Southwest Missouri St.',
 'Southwest Texas St.',
 'St. Francis NY',
 'St. Francis PA',
 'Tarleton St.',
 'Tennessee Martin',
 'Texas A&M Corpus Chris',
 'Texas Pan American',
 'UT Rio Grande Valley',
 'Winston Salem St.'}

In [8]:
merged_df.to_csv(f"{OUTPUT_DATA_DIR}/kenpom.csv", index=False)